# <b><i> This is the 1st part of the Week 3 Coursera Capstone Project for the IBM Data Science Professional Certificate </i></b>

<b>We use BeautifulSoup and urllib packages to scrape and make a request to the wiki page. We use lxml parser to parse the page into BeautifulSoup Object that contains the tags and use the table tag to get the content.</b>

In [1]:
!pip install bs4

In [2]:
#import all relevant libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
#url import
url = urlopen(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
#create BeautifulSoup object
soup = BeautifulSoup(url, "lxml")

In [5]:
table = soup.table

<b>Next, we loop through each row in the table and get the data from each row appended into lists.</b>

In [6]:
#create lists and append relevant data from table
PostalCode=[]
Borough=[]
Neighborhood=[]

for tr in table.find_all('tr')[1:]:
    PostalCode.append(tr.find_all('td')[0].get_text())
    Borough.append(tr.find_all('td')[1].get_text())
    Neighborhood.append(tr.find_all('td')[2].get_text().strip('\n'))
    

<b>Next step is creating a Dataframe.</b>

In [7]:
#create dataframe with the previously created lists
df = pd.DataFrame({
        'PostalCode':PostalCode,
        'Borough':Borough,
        'Neighborhood':Neighborhood
    })

#print to check results
df.head()

,PostalCode,Borough,Neighborhood
0,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North
1,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park)
2,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned
3,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned
4,\nM1HScarborough(Cedarbrae)\n\n,\nM2HNorth York(Hillcrest Village)\n\n,M3HNorth York(Bathurst Manor / Wilson Heights ...


<b>Now we need to transform and append the data, while also assigning possible "Not assigned" Neighborhoods with the borough data.</b>

In [8]:
#transform data
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#import transformed data into DataFrame
df=pd.DataFrame(table_contents)

<b><i>Checking to see if the transformation went as planned.</i></b>

In [9]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [10]:
df.shape

(103, 3)

# <b><i>This is the 2nd part of the Week 3 Coursera Capstone Project for the IBM Data Science Professional Certificate</i></b>

<b>Import coordinates and merge data</b>

In [11]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)

df_data_1.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_data = pd.merge(df,df_data_1, on='PostalCode')

df_data.head(10)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<i>As we can see, the new Dataframe has all the required information.</i>

# <b><i>This is the 3rd part of the Week 3 Coursera Capstone Project for the IBM Data Science Professional Certificate</i></b>

<b>This part focuses on clustering the neighborhoods of Toronto.</b>

<b>New imports for maps and clustering, from the previous lab setup.</b>

In [12]:
import json # library to handle JSON files
import numpy as np

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1


In [13]:
#create a map of Toronto
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<b>We will only explore Toronto boroughs.</b>

In [15]:
toronto = df_data.loc[df_data['Borough'].str.contains('Toronto')].reset_index()
toronto

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,35,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106


Lets explore the Central Bay Street in Downtown Toronto

In [16]:
toronto.loc[5, 'Neighborhood']

'Central Bay Street'

In [17]:
neighborhood_latitude = toronto.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


Now lets continue with the predefined lab setup

CLIENT_ID=
CLIENT_SECRET =

<b>Now let's explore the top 10 venues in a radius of 500 meters</b>

In [20]:
radius=500
LIMIT=10
VERSION='20190915'

url_1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Now we send the get request and examine the results.

In [21]:
results = requests.get(url_1).json()
results

{'meta': {'code': 200, 'requestId': '60981028a3a92d66b46f7384'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 64,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

Now let's get the category type and clean the results.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,Tim Hortons,Coffee Shop,43.658570,-79.385123
4,NEO COFFEE BAR,Coffee Shop,43.660130,-79.385830


<b>Now we create a function to repeat the process for every neighborhood in Toronto, as was done in the lab exercise.</b>

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url_2).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

<b>Afterwards we need to analyse each neighborhood before we can get to clustering.
Every step is found in the lab exercise with the New York neighborhoods.</b>

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,...,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arts & Crafts Store,...,Swim School,Tailor Shop,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.1,0.00,0.0,0.0,0.0
6,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,"Dufferin, Dovercourt Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0


Now we are going to print each neighborhood with their top 5 most common venues.

In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                 Farmers Market   0.2
1                           Park   0.1
2                     Restaurant   0.1
3  Vegetarian / Vegan Restaurant   0.1
4                         Museum   0.1


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop   0.2
1               Pet Store   0.1
2          Breakfast Spot   0.1
3                     Gym   0.1
4  Furniture / Home Store   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge   0.2
1            Boutique   0.1
2  Airport Food Court   0.1
3        Airport Gate   0.1
4     Airport Service   0.1


----Central Bay Street----
                        venue  freq
0                 Coffee Shop   0.4
1          Italian Restaurant   0.1
2                   Gastropub   0.1
3  Modern European Restaurant   0.1
4   M

<b>Now we put into a pandas Dataframe and display the top 10 venues for each neighborhood.</b>

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Park,Restaurant,Vegetarian / Vegan Restaurant,Museum,Cocktail Bar,Beer Bar,Seafood Restaurant,Liquor Store,Jewelry Store
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Pet Store,Breakfast Spot,Gym,Furniture / Home Store,Italian Restaurant,Café,Bar,Bakery,Middle Eastern Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Boutique,Airport Food Court,Airport Gate,Airport Service,Airport,Harbor / Marina,Rental Car Location,Bar,Middle Eastern Restaurant
3,Central Bay Street,Coffee Shop,Italian Restaurant,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Park,Spa,Harbor / Marina,Korean Restaurant,Gift Shop
4,Christie,Café,Grocery Store,Candy Store,Italian Restaurant,Restaurant,Coffee Shop,Movie Theater,Museum,Jewelry Store,Modern European Restaurant


<b>Now for the clustering part. First we need to define the number of clusters.</b>

In [56]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 0, 1, 2, 1, 2, 2, 1], dtype=int32)

Let's put it into a pandas dataframe together with the top 10 most common venues.

In [57]:
neighborhoods_venues_sorted.insert(0, 'Clusters Labels', kmeans.labels_)

In [58]:
toronto_merged = toronto

# merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Clusters Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Historic Site,Chocolate Shop,Restaurant,Distribution Center,Breakfast Spot,Bakery,Spa,Park,Gym / Fitness Center
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Pizza Place,Comic Shop,Music Venue,Theater,Clothing Store,Plaza,Burrito Place,Ramen Restaurant,Mexican Restaurant
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Food Truck,Cosmetics Shop,Creperie,Middle Eastern Restaurant,Restaurant,Hotel,Japanese Restaurant,Gym,Historic Site
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Pub,Health Food Store,Yoga Studio,Italian Restaurant,Museum,Movie Theater,Modern European Restaurant,Middle Eastern Restaurant,Mexican Restaurant
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Farmers Market,Park,Restaurant,Vegetarian / Vegan Restaurant,Museum,Cocktail Bar,Beer Bar,Seafood Restaurant,Liquor Store,Jewelry Store


<b>It's time to create the map.</b>

In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Clusters Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<B>Thank you for your time</B>